In [ ]:
if len(groups_desc) > 0:
    markdown_str = ["## Differential Feature functioning"]
    markdown_str.append("This section shows differential feature functioning (DFF) plots "
                        "for all features and subgroups. The features are shown after applying "
                        "transformations (if applicable) and truncation of outliers.")
    display(Markdown('\n'.join(markdown_str)))

In [ ]:
# check if we already created the merged file in another notebook

try:
    df_train_preproc_merged
except NameError:
    df_train_preproc_merged = pd.merge(df_train_preproc, df_train_metadata, on = 'spkitemid')

for group in groups_desc:
    display(Markdown("### DFF by {}".format(group)))
    selected_columns = ['spkitemid', 'sc1'] + features_used + [group]
    df_melted = pd.melt(df_train_preproc_merged[selected_columns], id_vars=['spkitemid', 'sc1', group], var_name='feature')
    group_values = sorted(df_melted[group].unique())
    colors = sns.color_palette("Greys", len(group_values))
    with sns.axes_style('whitegrid'), sns.plotting_context('notebook', font_scale=1.2):
        p = sns.catplot(x='sc1', y='value', hue=group, hue_order = group_values,
                        col='feature', col_wrap=3, height=3, scale=0.6, palette=colors,
                        sharey=False, sharex=False, legend=False, kind="point",
                        data=df_melted)

        for i, axis in enumerate(p.axes):
            axis.set_xlabel('score')
            if i == 2:
                legend = axis.legend(group_values, title=group, 
                                     frameon=True, fancybox=True, 
                                     ncol=1, fontsize=10,
                                     loc='upper right', bbox_to_anchor=(1.75, 1))
                for j in range(len(group_values)):
                    legend.legendHandles[j].set_color(colors[j])
                plt.setp(legend.get_title(), fontsize='x-small')

        plt.tight_layout(h_pad=1.0)
        if use_thumbnails:
            imgfile = join(figure_dir, '{}_dff_{}.svg'.format(experiment_id, group))
            plt.savefig(imgfile)
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()